In [1]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)
3.5.2 (default, Nov 17 2016, 17:05:23) 
[GCC 5.4.0 20160609]


In [2]:
# Import dataset
# Python 2
# pip install jieba
# pip install pyldavis

# -*- coding:utf-8 -*-
# 20200 /SSD2/jack/pythonEnv/python2/bin/jupyter notebook 

import pandas as pd
df = pd.read_csv("cleaned.csv")
# df = pd.read_csv("datascience.csv", encoding='gb18030')

df.head()

,content
0,老 用户 有 特权 恭喜 您 获得 光纤 宽带 不 加价 升 资格 年底 前 凭 机主 有效...
1,近期 有 市民 手机 接到 内容 涉及 含 同学聚会 揭发 隐私 等 内容 要求 点击 链接...
2,您 有 一份 流量 大礼 未 领取 亲 开通 功能 天内 可以 领取 超大 广州 本地 免费...
3,尊敬 的 客户 您 已 成功 开通 中国电信 总机 服务 业务 你 的 短 号码 为 感谢您...
4,平安 豪礼 提前 送 苹果 送到 手 欢 大礼 场 眼镜 各种 优惠券 等 你 扫走 快 戳...


In [3]:
df.shape

(171993, 1)

In [4]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
# df["content_cutted"] = df#.apply(chinese_word_cut)

df["content_cutted"] = df.content.apply(chinese_word_cut)
df.content_cutted.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.890 seconds.
Prefix dict has been built succesfully.


0    老   用户   有   特权   恭喜   您   获得   光纤   宽带   不   ...
1    近期   有   市民   手机   接到   内容   涉及   含   同学聚会   揭...
2    您   有   一份   流量   大礼   未   领取   亲   开通   功能   ...
3    尊敬   的   客户   您   已   成功   开通   中国电信   总机   服务...
4    平安   豪礼   提前   送   苹果   送到   手   欢   大礼   场   ...
Name: content_cutted, dtype: object

In [5]:
df.content_cutted.shape

(171993,)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [7]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20
# n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0
                               )
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [8]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [9]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
服务 尊敬 客户 满意 回复 客服 进行 本次 业务 拨打 您好 评价 中国电信 短信 感谢您 免费 办理 我们 热线 数字
Topic #1:
家长 孩子 老师 完成 今天 请勿 百度 开始 您好 准备 检查 来自 旅游 复习 督促 同学 上课 第一 车辆 没有
Topic #2:
通知 邮件 计算 提醒 安全 教育 电子 验证 学校 机会 账号 时间 单元 报名 公布 物流 他人 中学 收益 不要
Topic #3:
流量 中国移动 回复 开通 使用 套餐 本月 订购 提醒 小时 全国 办理 推荐 自动 每月 话费 业务 立即 省内 剩余
Topic #4:
邮箱 回复 资费 标准 科技 收费 内容 定制 消息 实验 浙江 幼儿园 发财树 主人 知道 陌陌 互动 一笔 点播 以下
Topic #5:
查看 银行 提示 温馨 金融 优步 保险 分期 标准计 江苏 发件人 入住 正文 再次 外卖 大众 我行 点评 周一 访问
Topic #6:
作业 工作 出行 中国 各位 游戏 下午 商品 上午 安排 信号 预计 今晚 做好 集团 健康 快乐 保持 你好 天气
Topic #7:
验证码 微信 关注 密码 明天 公众 欢迎 客服 输入 帐号 绑定 修改 进入 证券 手机号 派送 基金 下单 动态 申购
Topic #8:
操作 注册 申请 注意 忽略 现金 学生 短信 如非 还款 信用卡 以上 查收 完成 周末 购物 额度 一次 一份 良好
Topic #9:
快递 阅读 中心 确认 财富 即将 发货 大家 提供 选择 签收 付款 百世 取件 快件 疑问请 网络 北京 拒收 速递
Topic #10:
移动 联系 电话 本人 手机 尽快 问题 您好 谢谢 时间 信息 及时 进行 预约 平台 要求 公司 通过 系统 使用
Topic #11:
账单 点击 详情 登录 积分 订单 下载 直接 查询 话费 客户端 月话费 成功 手机 了解 登陆 取消 天天 详情请 退款
Topic #12:
用户 尊敬 小学 您好 数学 语文 学习 参加 英语 及时 费用 缴费 情况 联通 听写 用于 课堂 社区 为准 首次
Topic #13:
活动 短信 领取 参与 手机 免费 收到 点击 即可 官方 福利 转发 现在 优惠券 立刻 广东 多多 邀请 每日 深圳

In [11]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/SSD2/jack/pythonEnv/python3/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
19     9.566854        1       1 -0.220982  0.036273
10     8.229819        1       2 -0.040493 -0.131964
1      7.432693        1       3  0.096095 -0.035644
11     6.410534        1       4  0.051989  0.254310
2      6.231469        1       5  0.091565 -0.128323
0      5.691182        1       6 -0.269448 -0.034837
3      5.545306        1       7 -0.180665 -0.017948
7      5.365464        1       8  0.081101  0.050637
6      5.123402        1       9  0.085846 -0.086222
13     4.799544        1      10 -0.057754  0.296313
18     4.501450        1      11 -0.052322 -0.124055
16     4.221984        1      12  0.100076 -0.105476
17     4.056967        1      13 -0.006097  0.065673
8      3.966841        1      14  0.088752  0.154056
9      3.604754        1      15  0.113481 -0.070773
12     3.571305        1      16 -0.102412 -0.113420
4      3.372994        1      17 -0.186657  0.002262
15     3.223575        1      18  0.127116 -0.119829
5      3.129560        1      19  0.142634  0.047376
14     1.954305        1      20  0.138173  0.061591, topic_info=     Category          Freq  Term         Total  loglift  logprob
term                                                             
925   Default  41562.000000    退订  41562.000000  30.0000  30.0000
998   Default  27008.000000   验证码  27008.000000  29.0000  29.0000
313   Default  35491.000000    回复  35491.000000  28.0000  28.0000
659   Default  21958.000000    流量  21958.000000  27.0000  27.0000
940   Default  18186.000000    邮箱  18186.000000  26.0000  26.0000
618   Default  16104.000000    查看  16104.000000  25.0000  25.0000
393   Default  18333.000000    家长  18333.000000  24.0000  24.0000
398   Default  23575.000000    尊敬  23575.000000  23.0000  23.0000
879   Default  14920.000000    账单  14920.000000  22.0000  22.0000
676   Default  17455.000000    点击  17455.000000  21.0000  21.0000
930   Default  15930.000000    通知  15930.000000  20.0000  20.0000
367   Default  14789.000000    孩子  14789.000000  19.0000  19.0000
468   Default  11017.000000    快递  11017.000000  18.0000  18.0000
599   Default  12709.000000    服务  12709.000000  17.0000  17.0000
708   Default  10467.000000    用户  10467.000000  16.0000  16.0000
461   Default  11532.000000    微信  11532.000000  15.0000  15.0000
385   Default  11732.000000    客户  11732.000000  14.0000  14.0000
859   Default  11976.000000    详情  11976.000000  13.0000  13.0000
54    Default  10888.000000  中国移动  10888.000000  12.0000  12.0000
938   Default  11227.000000    邮件  11227.000000  11.0000  11.0000
744   Default  14548.000000    短信  14548.000000  10.0000  10.0000
881   Default   9420.000000    账户   9420.000000   9.0000   9.0000
657   Default   9878.000000    活动   9878.000000   8.0000   8.0000
758   Default  11624.000000    移动  11624.000000   7.0000   7.0000
728   Default  10524.000000    登录  10524.000000   6.0000   6.0000
186   Default   9047.000000    关闭   9047.000000   5.0000   5.0000
890   Default   8375.000000    资费   8375.000000   4.0000   4.0000
622   Default   8329.000000    标准   8329.000000   3.0000   3.0000
757   Default   9955.000000    积分   9955.000000   2.0000   2.0000
206   Default   8608.000000    分钟   8608.000000   1.0000   1.0000
...       ...           ...   ...           ...      ...      ...
419   Topic20   5021.453917    已经   5022.437948   3.9349  -2.0726
624   Topic20   2347.084035    校园   2348.068066   3.9347  -2.8331
214   Topic20   2332.237234    到达   2333.221265   3.9347  -2.8394
480   Topic20   1824.292783    成为   1825.276813   3.9346  -3.0851
119   Topic20   1753.857852    企业   1754.841883   3.9346  -3.1245
671   Topic20   1684.158289    湖南   1685.142319   3.9346  -3.1650
222   Topic20   1671.205605    加入   1672.189636   3.9345  -3.1727
148   Topic20   1471.567557    保障   1472.551588   3.9345  -3.2999
165   Topic20   1356.511093    先生   1357.495123   3.9344  -3.3814
847   Topic20   121